# Self Attention

In [1]:
import torch

# Say, we have a input sequence of 6 word and each word/token has 3 dimension embedding.
# So, the input sequence is of shape (6, 3)
inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89], # word 1, Your (x^1)
        [0.55, 0.87, 0.66], # word 2, journey (x^2)
        [0.57, 0.85, 0.64], # word 3, starts (x^3)
        [0.22, 0.58, 0.33], # word 4, with (x^4)
        [0.77, 0.25, 0.10], # word 5, one (x^5)
        [0.05, 0.80, 0.55], # word 6, step (x^6)
    ]
)

# Calculate the Context Embedding (For one query)

## Calculate the Attention Score Vector

In [2]:
query = inputs[1] # query vector, x^2
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(query, x_i)  # dot product of query and each word/token
print(attn_scores_2) # This is the weight score vector for query x^2

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


## Calculate the Attention Weights (Use Sum for Normalization)

In [3]:
attn_weight_2 = attn_scores_2 / attn_scores_2.sum()

print("Attention weights:", attn_weight_2)
print("Sum:", attn_weight_2.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


## Calculate the Attention Weights (Use Softmax for Normalization)

In [4]:
attn_weight_2 = torch.nn.functional.softmax(attn_scores_2, dim=0)
print("Attention weights:", attn_weight_2)

# Or
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)
attn_weight_2 = softmax_naive(attn_scores_2)
print("Attention weights:", attn_weight_2)

print("Sum:", attn_weight_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


## Calculate the Context Embedding

In [5]:
context_embedding_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_embedding_2 += attn_weight_2[i] * x_i
    
print("Context embedding:", context_embedding_2)

Context embedding: tensor([0.4419, 0.6515, 0.5683])


# Calculate the Context Embedding (For a sequence of queries)

In [6]:
# Create a matrix of Attention Scores
attn_scores = torch.empty(inputs.shape[0], inputs.shape[0])

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)  # dot product of each word/token with each other

print("Attention scores matrix:\n", attn_scores)

# Or, Calculate the Attention Score matrix by using matrix multiplication
attn_scores = torch.mm(inputs, torch.t(inputs)) # Input matrix x Input matrix transpose

print("Attention scores matrix:\n", attn_scores)

# Or, a simplified presentation of matrix multiplication
attn_scores = inputs @ inputs.T

print("Attention scores matrix:\n", attn_scores)

Attention scores matrix:
 tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])
Attention scores matrix:
 tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])
Attention scores matrix:
 tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0

## Calculate the Attention Weights (Use Softmax for Normalization)

In [7]:
attn_weights = torch.softmax(attn_scores, dim=1)
print("Attention weights:\n", attn_weights)

Attention weights:
 tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


## Calculate the Context Embedding

In [8]:
context_embeddings = attn_weights @ inputs
print("Context embeddings:\n", context_embeddings)

# Double check the context embedding for query x^2
print("Context embedding for query x^2:", context_embeddings[1])
print("Context embedding for query x^2 (naive):", context_embedding_2)

Context embeddings:
 tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])
Context embedding for query x^2: tensor([0.4419, 0.6515, 0.5683])
Context embedding for query x^2 (naive): tensor([0.4419, 0.6515, 0.5683])


# Scaled Dot-Product Attention

## Create Weight Matrices for the Query, Key and Value (For one Query)

In [9]:
x_2 = inputs[1] # query vector, x^2
# Define sizes of the weight parameter matrices (query, key, value)
d_in = inputs.shape[1]   # This is dimension of the input token
d_out = inputs.shape[1] 

In [10]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [11]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print("Query:", query_2)

keys = inputs @ W_key
values = inputs @ W_value
print("Key:", keys)
print("Value:", values)

Query: tensor([0.8520, 0.4161, 1.0138])
Key: tensor([[0.6813, 0.2706, 1.0793],
        [0.7305, 0.4227, 1.1993],
        [0.7355, 0.4227, 1.1901],
        [0.3363, 0.2225, 0.6077],
        [0.6184, 0.3038, 0.6909],
        [0.3178, 0.2383, 0.7426]])
Value: tensor([[0.4976, 0.9655, 0.7614],
        [0.9074, 1.3518, 1.5075],
        [0.8976, 1.3391, 1.4994],
        [0.5187, 0.7319, 0.8493],
        [0.4699, 0.7336, 0.9307],
        [0.6446, 0.9045, 0.9814]])


## Calculate the Attention Scores

In [12]:
attn_scores_2 = query_2 @ keys.T
print("Attention scores:", attn_scores_2)

Attention scores: tensor([1.7872, 2.0141, 2.0091, 0.9952, 1.3538, 1.1227])


## Calculate the Attention Weights (Use Softmax for Normalization)

In [13]:
d_k = keys.shape[-1]

attn_weight_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)

print("Attention weights:", attn_weight_2)

Attention weights: tensor([0.1862, 0.2123, 0.2117, 0.1179, 0.1450, 0.1269])


## Calculate the Context Embedding

In [14]:
context_embedding_2 = attn_weight_2 @ values
print("Context embedding:", context_embedding_2)

Context embedding: tensor([0.6864, 1.0577, 1.1389])


# Implementing a Scaled Dot-Product Attention Class

In [15]:
# Use nn.Parameter to create weight matrices
import torch.nn as nn

class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super(SelfAttention_v1, self).__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
        self.W_key = nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
        self.W_value = nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
        
    def forward(self, inputs):
        query = inputs @ self.W_query
        key = inputs @ self.W_key
        value = inputs @ self.W_value
        
        attn_scores = query @ key.T
        attn_weights = torch.softmax(attn_scores / key.shape[-1]**0.5, dim=-1)
        
        context_embedding = attn_weights @ value
        return context_embedding

In [16]:
# Test the SelfAttention_v1 class
torch.manual_seed(123)

d_in = inputs.shape[1]
d_out = 2

self_attn_v1 = SelfAttention_v1(d_in, d_out)
context_embedding_v1 = self_attn_v1(inputs)
print("Context embeddings:\n", context_embedding_v1)

Context embeddings:
 tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]])


In [17]:
# Use nn.Linear to create weight matrices
import torch.nn as nn

class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        
        self.attn_weights = None

    def forward(self, inputs):
        query = self.W_query(inputs)
        key = self.W_key(inputs)
        value = self.W_value(inputs)

        attn_scores = query @ key.T
        self.attn_weights = torch.softmax(attn_scores / key.shape[-1]**0.5, dim=-1)

        context_embedding = attn_weights @ value
        return context_embedding

In [18]:
# Test the SelfAttention_v2 class
torch.manual_seed(789)

d_in = inputs.shape[1]
d_out = 2

self_attn_v2 = SelfAttention_v2(d_in, d_out)
context_embedding_v2 = self_attn_v2(inputs)
print("Context embeddings:\n", context_embedding_v2)

Context embeddings:
 tensor([[-0.0858,  0.0607],
        [-0.0921,  0.0578],
        [-0.0912,  0.0588],
        [-0.0885,  0.0572],
        [-0.0689,  0.0845],
        [-0.0973,  0.0461]], grad_fn=<MmBackward0>)


In [19]:
# Transfer the weight matrices from SelfAttention_v2 to SelfAttention_v1
print("Before transfer - V1 Query:", self_attn_v1.W_query)
print("Before transfer - V2 Query:", self_attn_v2.W_query.weight)

print("Before transfer - V1 Key:", self_attn_v1.W_key)
print("Before transfer - V2 Key:", self_attn_v2.W_key.weight)

print("Before transfer - V1 Value:", self_attn_v1.W_value)
print("Before transfer - V2 Value:", self_attn_v2.W_value.weight)



Before transfer - V1 Query: Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]])
Before transfer - V2 Query: Parameter containing:
tensor([[ 0.3161,  0.4568,  0.5118],
        [-0.1683, -0.3379, -0.0918]], requires_grad=True)
Before transfer - V1 Key: Parameter containing:
tensor([[0.1366, 0.1025],
        [0.1841, 0.7264],
        [0.3153, 0.6871]])
Before transfer - V2 Key: Parameter containing:
tensor([[ 0.4058, -0.4704,  0.2368],
        [ 0.2134, -0.2601, -0.5105]], requires_grad=True)
Before transfer - V1 Value: Parameter containing:
tensor([[0.0756, 0.1966],
        [0.3164, 0.4017],
        [0.1186, 0.8274]])
Before transfer - V2 Value: Parameter containing:
tensor([[ 0.2526, -0.1415, -0.1962],
        [ 0.5191, -0.0852, -0.2043]], requires_grad=True)


# Causal Attention

In [21]:
attn_weight_2 = self_attn_v2.attn_weights

print("Attention weights:\n", attn_weight_2)

Attention weights:
 tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [22]:
# Create a mask for the lower triangular part of the attention matrix
length = inputs.shape[0]  # length of the input sequence
mask = torch.tril(torch.ones(length, length), diagonal=0)

print("Mask:\n", mask)

Mask:
 tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [24]:
# Apply the mask to the attention weights
attn_weight_2_masked = attn_weight_2 * mask

print("Attention weights (masked):\n", attn_weight_2_masked)

Attention weights (masked):
 tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)
